In [ ]:
import os
os.getcwd()

In [ ]:
import arcpy

# arcpy.SignInToPortal()

arcpy.env.overwriteOutput = True
arcpy.env.workspace = '00_input'

In [ ]:
# !pip install gcsfs

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# os.getenv("GOOGLE_CLOUD_PROJECT")

In [ ]:
import gcsfs
import geopandas as gpd
import pandas as pd

gcs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='../../backend/user_id.json')
fs = gcs


In [ ]:
#### DS TRAIN CHECK RESAMPLING
import os
import geopandas as gpd
import pandas as pd
import gcsfs

path_training = '00_input/training_shp/'

layers = [shp for shp in os.listdir(path_training) if shp.endswith('.shp') and shp.startswith('sample')]
gcs_path = 'gs://remote_sensing_saas/01-korindo/sample_tsfresh/20251112_training_gdf_col_filtered.parquet'

use_parquet_training = True

if use_parquet_training != True:
    gdf_list = []

    for lyr in layers:
        print(lyr)
    gdf = gpd.read_file(os.path.join(path_training,lyr))
    gdf['layer'] = lyr.replace('.shp', '')
    print(gdf.crs)
    gdf_utm = gdf.to_crs(crs_ds)
    gdf_list.append(gdf_utm)

    training_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
    # training_gdf = gpd.read_file('00_input/training_shp/sample_1.shp')
    # training_gdf.head()
    # training_gdf.head()

    # training_gdf.crs

    # training_gdf.geometry.head()
    # training_gdf.columns

    list_columns_time = [i for i in training_gdf.columns if i.startswith('t_') and not i.endswith('D')]
    list_columns_time = list(sorted(list_columns_time))
    # list_columns_time

    columns_filter = ['layer'] + list_columns_time + ['geometry']

    training_gdf_col_filtered = training_gdf[columns_filter]
    # training_gdf_col_filtered.head()
    # training_gdf_col_filtered.plot()

    print(training_gdf_col_filtered.shape)

    # Save as GeoParquet (BEST option)
    gcs_path = gcs_path    
    training_gdf_col_filtered.to_parquet(gcs_path, filesystem=fs, compression='snappy')

else:
    training_gdf_col_filtered = gpd.read_parquet(gcs_path, filesystem=fs)

In [ ]:
training_gdf_col_filtered.head()

In [ ]:
import pandas as pd
import geopandas as gpd

use_existing_df_long = False # save some cost here to download from gcs
#gcs_path_df_long = 'gs://remote_sensing_saas/01-korindo/sample_tsfresh/20251112_df_long.parquet'

if use_existing_df_long != True:
    # training_gdf_col_filtered

    t_cols = [col for col in training_gdf_col_filtered.columns if col.startswith('t_')]

    # Melt the dataframe with geometry as id_var
    df_long = pd.melt(
        training_gdf_col_filtered, 
        id_vars=['layer','geometry'],
        value_vars=t_cols,
        var_name='time_period',
        value_name='value'
    )

    # Set geometry as index
    # df_long = df_long.set_index('geometry')

    print(f"Original shape: {training_gdf_col_filtered.shape}")
    print(f"Long format shape: {df_long.shape}")
    print(f"Time columns found: {len(t_cols)}")
    # print(f"\nFirst few rows:")
    # df_long.head()
    # df_long.layer.unique()

    # df_long.reset_index(inplace=True)
    # df_long.head()

    ## reformating the column (date)
    df_long = df_long.rename(columns={'value': 'type'})
    df_long['date'] = df_long['time_period'].str[2:].astype(int)
    # df_long.head()

    # type(df_long)

    training_gdf = df_long.copy()
    # Remove multipolygons
    # training_gdf = training_gdf.explode()
    print(training_gdf.crs)

    ##### conversion
    # Drop rows where 'type' is NA
    training_gdf = training_gdf.copy()
    training_gdf = training_gdf.dropna(subset=['type'])
    training_gdf['date'] = pd.to_datetime(training_gdf['date'], format='%Y%m')

    # add year columnt
    training_gdf['year'] = training_gdf['date'].dt.year

    # convert column 'type' from string to int
    training_gdf['type'] = training_gdf['type'].astype(int)
    print('shape of training_gdf after drop NA:', training_gdf.shape)
    
    # training_gdf.to_parquet(gcs_path_df_long, filesystem=fs, compression='snappy') # i shouldn't export for now

else:
    training_gdf = gpd.read_parquet(gcs_path_df_long, filesystem=fs)

In [ ]:
training_gdf.head()

In [ ]:
# ## DISSOLVING, takes too long:
# gdf_1_dissolved = training_gdf[training_gdf['type'] == 1].dissolve(by=['date', 'layer'])
# gdf_0_dissolved = training_gdf[training_gdf['type'] == 0].dissolve(by=['date', 'layer'])

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().resolve().parents[1]  # adjust if your cwd differs
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from arcpy_arcgis_api_lib.xls_append.data_val import pandas_to_esri

In [ ]:
import arcpy
import geopandas as gpd
from pathlib import Path
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcpy_arcgis_api_lib.xls_append.data_val import pandas_to_esri

def export_training_gdf_to_feature_class(
    training_gdf: gpd.GeoDataFrame,
    gdb_name: str = "training_timeseries.gdb",
    feature_class_name: str = "training_timeseries",
    workspace: str | None = None,
    overwrite: bool = True,
) -> str:
    """
    Persist `training_gdf` into a file geodatabase feature class.

    - Creates the target GDB inside `workspace` (default: `arcpy.env.workspace`)
    - Uses `arcgis.features.GeoAccessor` for conversion, so geometry/CRS are preserved
    - Leverages `arcpy_arcgis_api_lib`’s `pandas_to_esri` mapping to preview ArcGIS field types
    """
    if not isinstance(training_gdf, gpd.GeoDataFrame):
        raise TypeError("training_gdf must be a GeoDataFrame")

    if training_gdf.crs is None:
        raise ValueError("training_gdf is missing a CRS; set one before exporting")

    base_workspace = workspace or arcpy.env.workspace or arcpy.env.scratchWorkspace
    if base_workspace is None:
        raise ValueError("Set `arcpy.env.workspace` or pass a `workspace` path")

    gdb_path = Path(base_workspace) / gdb_name
    if gdb_path.suffix.lower() != ".gdb":
        gdb_path = gdb_path.with_suffix(".gdb")
    gdb_path.parent.mkdir(parents=True, exist_ok=True)

    if not arcpy.Exists(str(gdb_path)):
        arcpy.management.CreateFileGDB(str(gdb_path.parent), gdb_path.name)

    esri_type_preview = {
        col: pandas_to_esri.get(str(training_gdf[col].dtype), "esriFieldTypeString")
        for col in training_gdf.columns
        if col != training_gdf.geometry.name
    }
    display(pd.DataFrame.from_dict(esri_type_preview, orient="index", columns=["esri_type"]))

    sedf = GeoAccessor.from_geodataframe(training_gdf)
    output_fc = str(gdb_path / feature_class_name)

    if arcpy.Exists(output_fc) and overwrite:
        arcpy.management.Delete(output_fc)

    sedf.spatial.to_featureclass(location=output_fc)
    return output_fc

In [ ]:
feature_class_path = export_training_gdf_to_feature_class(
    training_gdf,
    gdb_name="training.gdb",
    feature_class_name="korindo_training_long"
)
print(f"Feature class saved to: {feature_class_path}")
print(arcpy.management.GetCount(feature_class_path))

In [ ]:
feature_class_path